In [ ]:
from mycelia.shared.model import load_base_model
from mycelia.shared.modeling.modeling_mycelia import get_base_tokenizer
from mycelia.config import MinerConfig
from mycelia.shared.datasets import get_dataloader
from mycelia.shared.app_logging import configure_logging, structlog
from mycelia.shared.helper import get_nested_attr
from transformers import (
    AutoConfig,
    AutoTokenizer,
    LlamaConfig,
    LlamaForCausalLM,
    OlmoForCausalLM,
    AutoModelForCausalLM,
    PretrainedConfig
)
configure_logging()

config = MinerConfigonfig()
config.model.device = "cuda:2"

/home/isabella/crucible/subnet-MoE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-10-08 23:32:26 [info     ] found existing config path /home/isabella/crucible/subnet-MoE/checkpoints/centralised/config.json [mycelia.config]
2025-10-08 23:32:26 [info     ] Keys present in self but missing in other at 'model': {'foundation'} [mycelia.config]
2025-10-08 23:32:26 [info     ] Bumped run_name to centralised-v2 due to config differences. [mycelia.config]


In [2]:
url = "http://localhost:8000/checkpoint"
token = "supersecrettoken"
timeout = 30
headers = {"Authorization": f"Bearer {token}"} if token else {}

response = requests.get(url, headers=headers, stream=True, timeout=timeout)
response.content

NameError: name 'requests' is not defined

In [2]:
config.ckpt.resume_from_ckpt = True
model, em = load_base_model(rank = 0, config = config)

2025-10-08 23:32:30 [info     ] rank 0: Checkpoint path /home/isabella/crucible/subnet-MoE/checkpoints/centralised-v2 not found, starting from scratch [mycelia.shared.checkpoint]
2025-10-08 23:33:23 [info     ] em init                      [mycelia.shared.expert_manager]
2025-10-08 23:33:23 [info     ] em get expert layer          [mycelia.shared.expert_manager]
2025-10-08 23:33:23 [info     ] em get expert layer 1        [mycelia.shared.expert_manager]
2025-10-08 23:33:23 [info     ] Detected expert layerss: [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27] [mycelia.shared.expert_manager]
2025-10-08 23:33:23 [info     ] seed 0 Expert group assignment for layer 27: {0: [1, 4, 5, 3], 1: [7, 2, 6, 0]} [mycelia.shared.expert_manager]


In [6]:
sum(p.numel() for p in model.parameters()) / 1e9

1.755588608

In [7]:
model.generate()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
Caching is incompatible with gradient checkpointing in DeepseekV3DecoderLayer. Setting `past_key_values=None`.
/home/isabella/crucible/subnet-MoE/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


tensor([[100000,  98475,  91773,  71992,  61337,  66154,  93447,  11056,  47337,
          39391,  20095,  78222,  46311,  43265,  77841,  26945,  49068,  89039,
          45949,   9281,  62924]], device='cuda:2')

In [2]:
model_org = AutoModelForCausalLM.from_pretrained(config.model.model_path, trust_remote_code=True) 
model_org.half().to(config.model.device)

Loading checkpoint shards:  29%|██▊       | 2/7 [00:02<00:07,  1.41s/it]


KeyboardInterrupt: 

In [4]:
rank = 0
tokenizer = get_base_tokenizer(config)
train_dataloader = get_dataloader(config, rank=rank, world_size=config.data.world_size, tokenizer=tokenizer)
data_iter = iter(train_dataloader)

In [5]:
batch = next(data_iter)
batch_device = {}

for k,v in batch.items():
    batch_device[k] = v.to(config.model.device)

output = model(**batch_device)
# output = model_org(**batch_device)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


In [6]:
output.loss

tensor(0.1030, device='cuda:2', grad_fn=<CompiledFunctionBackward>)

In [7]:
batch_device['input_ids'][0]

tensor([100001, 100001, 100001,  ...,    317,   8479,     13], device='cuda:2')

In [8]:
batch_device['labels'][0]

tensor([-100, -100, -100,  ...,  317, 8479,   13], device='cuda:2')

In [9]:
output.logits

tensor([[[ 17.7800,  18.6786,  19.1207,  ...,  -8.0841,  -7.9584,  -7.8377],
         [ 17.4785,  18.3606,  19.0278,  ...,  -8.0779,  -7.9781,  -7.8453],
         [ 17.4605,  18.2474,  19.1266,  ...,  -8.3010,  -8.2209,  -8.0728],
         ...,
         [ -4.5423,  -1.4848,  -4.1500,  ...,   1.6418,   1.4665,   1.6030],
         [ 31.1800,  22.0167,  16.6788,  ..., -11.3496, -10.7641, -10.9224],
         [ 25.8157,  22.7863,  17.9649,  ..., -12.7231, -12.3283, -12.1269]],

        [[ 16.8019,  17.1421,  18.3094,  ...,  -8.8904,  -8.8373,  -8.6426],
         [ 16.6314,  17.0933,  18.3151,  ...,  -8.8764,  -8.8338,  -8.6328],
         [ 16.6937,  17.2471,  18.3336,  ...,  -8.8157,  -8.7735,  -8.5739],
         ...,
         [  3.1221,   0.2283,  -0.7212,  ...,  -2.6192,  -2.3678,  -2.3329],
         [ 22.6576,  17.6196,  17.4180,  ...,  -4.6654,  -4.6236,  -5.0201],
         [ 14.5111,  12.8422,  15.9084,  ...,  -4.6031,  -4.5122,  -4.5149]],

        [[ 17.9822,  18.9494,  19.2880,  ...